# Machine Learning (Aprendizaje Automático)

# Keras

Keras es una librería de Python de código abierto, potente y fácil de usar para desarrollar y evaluar modelos de aprendizaje profundo (Deap Learning). Es básicamente un wraper de la librerías de cálculo numérico `Theano` y `TensorFlow` que permite definir y entrenar modelos de redes neuronales en solo unas pocas líneas de código.

## Datos
El primer paso es definir las funciones y clases que vamos usar. Usaremos `NumPy` para cargar el conjunto de datos y usaremos dos clases del módulo `Keras` para definir nuestro modelo.

In [ ]:
# Importamos las librerias
from numpy import loadtxt
# Para instalar keras, se debe de instalar tensorflow
from keras.models import Sequential
from keras.layers import Dense

Ahora podemos cargar el conjunto de datos.

Vamos a utilizar el conjunto de datos de diabetes [Dataset](https://github.com/jbrownlee/Datasets). Este es un conjunto de datos de aprendizaje automático estándar del repositorio de aprendizaje automático de UCI. Describe los datos de los registros médicos de los indigenas Pima (grupo indígena que vive en el estado de Arizona (Estados Unidos) y en los estados mexicanos de Sonora y Chihuahua) y si tuvieron una aparición de diabetes dentro de los cinco años.

Como tal, es un problema de clasificación binaria (aparición de diabetes como 1 o no como 0). Todas las variables de entrada que describen a cada paciente son numéricas. Esto hace que sea fácil de usar directamente con redes neuronales que esperan valores numéricos de entrada y salida, y es buen ejercicio de primera red neuronal usando `Keras`.

El conjunto de datos está disponible desde aquí:

In [ ]:
#Para descargar el archivo
import urllib.request as URL
import pathlib

#Direccion al archivo en github, debe de ser el raw
rawfile = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'
#intento cargar el archivo
file = pathlib.Path('pima-indians-diabetes.data.csv')
#diferentes pasos si el archivo existe o no
if file.exists():
    print ("File exist")
else:
    print ("File not exist \n Dounloading ...")
    URL.urlretrieve(rawfile, 'pima-indians-diabetes.data.csv')
    print("File", 'pima-indians-diabetes.data.csv downloaded')


Ahora cargargamos el archivo como una matriz de números usando la función `loadtxt()` de NumPy. Hay ocho variables de entrada y una variable de salida (la última columna).

Variables de entrada (X):

1. Número de veces embarazada
2. Concentración de glucosa en plasma a 2 horas en una curva de glucosa
3. Presión arterial diastólica (mm Hg)
4. Espesor del pliegue cutáneo del tríceps (mm)
5. Insulina sérica de 2 horas (mu U / ml)
6. Índice de masa corporal (peso en kg / (altura en m) ^ 2)
7. Función del árbol genealógico de la diabetes
8. Años de edad
9. Variables de salida (0 o 1)

Una vez que el archivo CSV se carga en la memoria, podemos dividir las columnas de datos en variables de entrada y salida. Vamos a dividir la matriz en dos matrices seleccionando subconjuntos de columnas usando el operador de segmento `NumPy` estándar `:`.

In [ ]:
# Cargamos el archivo, el separador son comas ","
dataset = loadtxt('pima-indians-diabetes.data.csv', delimiter=',')
# Separamos los datos en entrada (X) y salida (y)
#selecciono las primeras 8 columnas desde el índice 0 al índice 7 
X = dataset[:,0:8]
# selecciono la columna de salida (la novena variable) 
y = dataset[:,8]

#
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

## El modelo de Keras
Los modelos en Keras se definen como una secuencia de capas. Creamos un modelo secuencial y agregamos capas de una en una hasta que estemos satisfechos con la arquitectura de la red. 

Lo primero que se debe hacer es asegurarse de que la capa de entrada tenga el número correcto de entidades de entrada. Esto se puede especificar al crear la primera capa con el argumento `input_dim` y establecerlo en 8 para las 8 variables de entrada.

### ¿Cómo sabemos el número de capas y sus tipos?

Ésto es muy dificil de saber. Hay reglas y criterios metodológicos que podemos usar y, pero muchas veses, la mejor estructura para la red se encuentra a través de un proceso de prueba y error. 

Generalmente, es necesaria una red lo suficientemente grande para capturar la estructura del problema. En este ejemplo, usaremos una estructura de red completamente conectada con tres capas.

Las capas completamente conectadas se definen utilizando la clase `Dense`. Podemos especificar el número de neuronas o nodos en la capa como primer argumento y especificar la función de activación usando el argumento `activation`.

### Algunas funciones de activación 
1. Sigmoide (logística)\
    __Ventajas:__
      - La función es diferenciable.
      - La función es monótona pero la derivada de la función no lo es.

    __Desventajas:__
      - La derivada de la función no es monótona.
      - Cuando las entradas se vuelven pequeñas o grandes, la función se satura en 0 o 1, con una derivada extremadamente cercana a 0. Por lo tanto, casi no tiene gradiente para propagarse a través de la red, por lo que no queda casi nada para las capas inferiores (Gradiente de fuga).
      - Computacionalmente costoso, pues la función tiene una operación exponencial.
      - La salida no está centrada en cero.

<!-- begin figure -->
<div style="background-color:rgba(255, 255, 255, 255); text-align:center; vertical-align: middle; padding:40px 0;">
  <img src="Fig/sigmoid_and_deriv.png" width="800" height="400" hspace="10"/>
</div>
<!-- end figure -->

2. Tangente hiperbólica (Tanh):\
    __Ventajas:__
    - La salida de $\tanh$ está centrada en cero con un rango de -1 a 1.
    - La función es diferenciable.
    - La función es monótona.
  
    __Desventajas:__
    - La derivada de la función no es monótona.
    - Gradiente de fuga.
    - Computacionalmente costoso

Se define como
$$
\tanh(x)=\frac{e^{x}-e^{-x}}{e^{x}+e^{-x}}
$$

<!-- begin figure -->
<div style="background-color:rgba(255, 255, 255, 255); text-align:center; vertical-align: middle; padding:40px 0;">
  <img src="Fig/tanh_and_deriv.png" width="800" height="400" hspace="10"/>
</div>
<!-- end figure -->

3. Unidad lineal rectificada (ReLU): es la función de activación más utilizada en el aprendizaje profundo. La función devuelve 0 si la entrada es negativa, pero para cualquier entrada positiva, devuelve ese valor.\
    __Ventajas:__
    - La función relu es continua.
    - La salida de ReLU no tiene un valor máximo ( no está saturado ) y esto ayuda al Gradient Descent.
    - La función es muy rápida de calcular (comparar con Sigmoid y Tanh).
  
    __Desventajas:__
    - Durante el entrenamiento, algunas neuronas mueren efectivamente, lo que significa que dejan de producir algo que no sea 0. En algunos casos, puede encontrar que la mitad de las neuronas de su red están muertas, especialmente si utilizó una tasa de aprendizaje alta.



La función se define como:
$$
\left\{ \begin{array}{ccc}
0 & si & x\leq0\\
x & si & x>0
\end{array}\right.
$$

<!-- begin figure -->
<div style="background-color:rgba(255, 255, 255, 255); text-align:center; vertical-align: middle; padding:40px 0;">
  <img src="Fig/relu_and_deriv.png" width="800" height="400" hspace="10"/>
</div>
<!-- end figure -->

4. ReLU con fugas (Leaky ReLU): es una ReLU modificada, que tiene todas las propiedades de ReLU, y nunca tendrá un problema de ReLU moribundo . 

Leaky ReLU se define como:

$$
\left\{ \begin{array}{ccc}
0.01x & si & x<0\\
x & si & x>0
\end{array}\right.
$$

5. ReLU paramétrico con fugas (PReLU): es una variación de Leaky ReLU, donde $\alpha$ es aprendido durante el entrenamiento y que puede ser modificado por retropropagación como cualquier otro parámetro. Muy útil en conjuntos de datos con gran cantidad de características, pero en conjuntos de datos más pequeños se corre el riesgo de sobreajustar el conjunto de entrenamiento.

6. Unidades lineales exponenciales (ELU): es una variación de ReLU con una mejor salida para $x <0$. La función se define como:

$$
\left\{ \begin{array}{ccc}
\alpha (e^x-1) & si & x\leq 0\\
x & si & x>0
\end{array}\right.
$$

Usaremos la función de activación de _unidad lineal rectificada_(ReLU) en las dos primeras capas y la función _Sigmoide_ en la capa de salida.

En los comiensos de las redes neuronales las funciones de activación _Sigmoid_ y $\tanh$ eran las las más usadas para todas las capas. actualmete, se logra un mejor rendimiento utilizando la función de activación _ReLU_. 
Para la capa de salida usamos una función _sigmoide_ para asegurarnos de que la salida de la red esté entre 0 y 1 y sea fácil de asignar a una probabilidad de clase 1 o ajustar a una clasificación estricta de cualquier clase con un umbral predeterminado de 0.5.

En resumen:

- El modelo espera filas de datos con 8 variables (el argumento `input_dim = 8`)
- La primera capa oculta tiene 12 nodos y utiliza la función de activación _relu_.
- La segunda capa oculta tiene 8 nodos y utiliza la función de activación _relu_.
- La capa de salida tiene un nodo y usa la función de activación _sigmoidea_.

In [ ]:
# Definamos el modelo en keras
model = Sequential()
#La entrada al modelo se define como un argumento en la primera capa oculta. 
# al agregar la primera capa Dense está haciendo 2 cosas, 
# definiendo la entrada y la primera capa oculta.
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

## Compilación del modelo de Keras
Ahora que el modelo está definido, podemos compilarlo.

La compilación del modelo utiliza bibliotecas numéricas eficientes (backend) como _Theano_ o _TensorFlow_. El backend elige automáticamente la mejor manera de representar la red para entrenar y hacer predicciones para ejecutar en su hardware, como CPU o GPU o incluso cálculos distribuidos.

Al compilar, debemos especificar algunas propiedades adicionales requeridas al entrenar la red: 
- La función de costo o pérdida que se usará para evaluar un conjunto de pesos 
- El optimizador se usa para buscar entre diferentes pesos para la red
- Cualquier métrica opcional que nos gustaría recopilar e informar durante el entrenamiento.

En este caso, usaremos la entropía cruzada (cross entropy) como argumento de costo. Esta función es espesíficamente para problemas de clasificación binaria y se define en Keras como `binary_crossentropy`.

El optimizador que vamos a usar es el algoritmo de descenso por gradiente estocástico `adam`. Esta es una versión popular del descenso por gradientes porque ajusta automáticamente sus parámetros y da buenos resultados en una amplia gama de problemas.

Finalmente, debido a que es un problema de clasificación usamos la _precisión de la clasificación_, definida a través del argumento de `metrics`.

In [ ]:
# compilamos el model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Ajuste el modelo Keras
Hemos definido el modelo y lo hemos compilado para tener un cálculo eficiente. Ahora es el momento de entrenar el modelo usando algunos datos, esto se hace llamando a la función `fit()` en el modelo.

El entrenamiento ocurre durante _épocas_ (Epoch) y cada _época_ se divide en _lotes_ (Batch).

- _Epoch:_ cada vez que todas las filas del conjunto de datos de entrenamiento pasan por la red.
- _Batch:_ una o más muestras consideradas por el modelo dentro de una época antes de que se actualicen los pesos.

El proceso de entrenamiento se ejecutará durante un número fijo de iteraciones a través del las _épocas_, que debemos especificar usando el argumento `epochs`. También debemos establecer el número de filas del conjunto de datos que se consideran antes de que los pesos del modelo se actualicen dentro de cada época, lo que se denomina tamaño de lote y se establece mediante el argumento `batch_size`.

Para este problema, ejecutaremos una pequeña cantidad de épocas (150) y usaremos un tamaño de lote relativamente pequeño de 10.

In [ ]:
#Ajuste del modelo
model.fit(X_train, y_train, epochs=150, batch_size=10)

## Evaluando el modelo
Hemos entrenado la red neuronal en todo el conjunto de datos y podemos evaluar el rendimiento de la red en el conjunto de datos de testeo, esto solo nos dará una idea de qué tan bien hemos modelado el conjunto de datos.

Se puede evaluar el modelo en el conjunto de datos de testeo usando la función `evaluate()` del modelo y pasarle los datos de testeo. Esto generará una predicción para cada par de entrada y salida y recopilará puntajes, incluida la pérdida promedio y cualquier métrica que se haya configurado, como la precisión (en este caso).

La función `evaluate()` devolverá una lista con dos valores. El primero será el costo o pérdida del modelo en el conjunto de datos y el segundo será la precisión del modelo en el conjunto de datos. Solo nos interesa informar la precisión, por lo que ignoraremos el valor de la pérdida.

In [ ]:
# Evaluando el modelo en los datos de testeo
cost , accuracy = model.evaluate(X_test, y_test)

## Haciendo predicciones

Las predicciones se hacen llamaando la función `predict()` del modelo. Estamos usando una función de activación sigmoidea en la capa de salida, por lo que las predicciones serán una probabilidad en el rango entre 0 y 1. 

In [ ]:
# Haciendo predicciones
predictions = model.predict(X_test)
for i in range(10):
	print('%s => %.3f (expected %d)' % (X_test[i].tolist(), predictions[i], y_test[i]))
 

Podemos convertirlas fácilmente en una predicción binaria para esta tarea de clasificación redondeándolas.

In [ ]:
# Con aproximacion
rounded = [round(x[0]) for x in predictions]
for i in range(10):
	print('%s => %d (expected %d)' % (X_test[i].tolist(), rounded[i], y_test[i]))

También para un solo punto, teniendo cuidado de usar la forma correcta de entrada

In [ ]:
model.predict(X_test[0].reshape(1,8))